# Model Comparison & Decision Dashboard

**Purpose**: Transform experimental results into production decisions

**Models compared**:
- LFM2.5-Audio-1.5B (LiquidAI)
- Whisper-Large-V3 (OpenAI) - baseline

**Metrics**: Accuracy (WER/CER), Speed (RTF), Quality (MOS), Memory

---

This notebook loads results from all test notebooks and produces:
- Comparative scorecard
- Production recommendation
- Cost-performance analysis

In [ ]:
# === COMPARISON SETUP ===

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime

print("=== Model Comparison Dashboard ===")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"Models: LFM2.5-Audio vs Whisper-Large-V3")
print("✅ Comparison setup complete")

In [ ]:
# === LOAD RESULTS ===

results_dir = Path("data/results")
results = {}

# Expected result files from each notebook
result_files = {
    "asr_lfm": "asr_evaluation_lfm_results.json",
    "asr_whisper": "asr_evaluation_whisper_results.json",
    "tts_lfm": "tts_evaluation_lfm_results.json",
    "conversation_lfm": "conversation_analysis_lfm_results.json",
    "conversation_whisper": "conversation_analysis_whisper_results.json",
}

print("Loading results from all notebooks...")

for key, filename in result_files.items():
    filepath = results_dir / filename
    if filepath.exists():
        with open(filepath, "r") as f:
            results[key] = json.load(f)
        print(f"  ✓ {key}: {filepath.stat().st_size / 1024:.1f} KB")
    else:
        print(f"  ✗ {key}: Not found (run notebook first)")

print(f"\nLoaded {len(results)}/{len(result_files)} result files")

In [ ]:
# === COMPARATIVE METRICS ===


def extract_metrics(results, model_type, test_type):
    """Extract key metrics from results."""
    key = f"{test_type}_{model_type}"
    if key not in results:
        return None

    data = results[key]
    return {
        "wer": data.get("wer", float("nan")),
        "cer": data.get("cer", float("nan")),
        "latency_ms": data.get("latency_ms", float("nan")),
        "rtf": data.get("rtf", float("nan")),  # Real-Time Factor
        "memory_mb": data.get("memory_mb", float("nan")),
        "success": data.get("success", False),
    }


# Build comparison table
comparison_data = []

# ASR comparison
for model in ["lfm", "whisper"]:
    metrics = extract_metrics(results, model, "asr")
    if metrics:
        comparison_data.append(
            {
                "Model": "LFM2.5-Audio" if model == "lfm" else "Whisper-Large-V3",
                "Test": "ASR (speech-to-text)",
                "WER (%)": metrics["wer"] * 100,
                "CER (%)": metrics["cer"] * 100,
                "Latency (ms)": metrics["latency_ms"],
                "RTF": metrics["rtf"],
                "Memory (MB)": metrics["memory_mb"],
            }
        )

# Conversation comparison
for model in ["lfm", "whisper"]:
    metrics = extract_metrics(results, model, "conversation")
    if metrics:
        comparison_data.append(
            {
                "Model": "LFM2.5-Audio" if model == "lfm" else "Whisper-Large-V3",
                "Test": "Conversation (multi-speaker)",
                "WER (%)": metrics["wer"] * 100,
                "CER (%)": metrics["cer"] * 100,
                "Latency (ms)": metrics["latency_ms"],
                "RTF": metrics["rtf"],
                "Memory (MB)": metrics["memory_mb"],
            }
        )

df_comparison = pd.DataFrame(comparison_data)
print("=== Comparative Scorecard ===")
print(df_comparison.to_string(index=False))

# Save comparison table
comparison_path = results_dir / "model_comparison_table.json"
df_comparison.to_json(comparison_path, orient="records", indent=2)
print(f"\n✓ Comparison saved to {comparison_path}")

In [ ]:
# === PRODUCTION SCORECARD ===


def calculate_production_score(metrics):
    """Calculate production readiness score (0-100)."""
    scores = []

    # Accuracy score (WER: lower is better)
    if not np.isnan(metrics["wer"]):
        wer_score = max(0, 100 - metrics["wer"] * 100)  # 0% WER = 100 score
        scores.append(wer_score)

    # Speed score (RTF: lower is better, <1.0 = realtime)
    if not np.isnan(metrics["rtf"]):
        rtf_score = max(0, 100 - metrics["rtf"] * 50)  # RTF=0 = 100, RTF=2.0 = 0
        scores.append(rtf_score)

    # Memory score (lower is better)
    if not np.isnan(metrics["memory_mb"]):
        mem_score = max(0, 100 - metrics["memory_mb"] / 20)  # 0MB = 100, 2GB = 0
        scores.append(mem_score)

    return np.mean(scores) if scores else 0


# Calculate scores
production_scores = []

for model in ["lfm", "whisper"]:
    asr_metrics = extract_metrics(results, model, "asr")
    conv_metrics = extract_metrics(results, model, "conversation")

    if asr_metrics:
        asr_score = calculate_production_score(asr_metrics)
        production_scores.append(
            {
                "Model": "LFM2.5-Audio" if model == "lfm" else "Whisper-Large-V3",
                "Test": "ASR",
                "Production Score": asr_score,
                "Accuracy Grade": "A" if asr_score >= 80 else "B" if asr_score >= 60 else "C",
            }
        )

    if conv_metrics:
        conv_score = calculate_production_score(conv_metrics)
        production_scores.append(
            {
                "Model": "LFM2.5-Audio" if model == "lfm" else "Whisper-Large-V3",
                "Test": "Conversation",
                "Production Score": conv_score,
                "Accuracy Grade": "A" if conv_score >= 80 else "B" if conv_score >= 60 else "C",
            }
        )

df_scores = pd.DataFrame(production_scores)
print("=== Production Readiness Scorecard ===")
print(df_scores.to_string(index=False))

# Determine overall recommendation
if len(df_scores) > 0:
    best_model = df_scores.groupby("Model")["Production Score"].mean().idxmax()
    best_score = df_scores.groupby("Model")["Production Score"].mean().max()

    print(f"\n=== PRODUCTION RECOMMENDATION ===")
    print(f"🏆 Recommended: {best_model}")
    print(f"   Overall Score: {best_score:.1f}/100")

    if best_score >= 80:
        print("   ✅ Ready for production deployment")
    elif best_score >= 60:
        print("   ⚠️  Ready with monitoring required")
    else:
        print("   ❌ Not recommended for production")

In [ ]:
# === VISUALIZATION ===

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle("Model Comparison: LFM2.5-Audio vs Whisper-Large-V3", fontsize=14, fontweight="bold")

# Plot 1: WER Comparison
if "WER (%)" in df_comparison.columns:
    ax1 = axes[0, 0]
    for model in df_comparison["Model"].unique():
        model_data = df_comparison[df_comparison["Model"] == model]
        ax1.bar(model_data["Test"], model_data["WER (%)"], label=model, alpha=0.7)
    ax1.set_ylabel("WER (%)")
    ax1.set_title("Word Error Rate (lower is better)")
    ax1.legend()
    ax1.tick_params(axis="x", rotation=45)

# Plot 2: Speed Comparison
if "RTF" in df_comparison.columns:
    ax2 = axes[0, 1]
    for model in df_comparison["Model"].unique():
        model_data = df_comparison[df_comparison["Model"] == model]
        ax2.bar(model_data["Test"], model_data["RTF"], label=model, alpha=0.7)
    ax2.axhline(y=1.0, color="r", linestyle="--", label="Realtime threshold")
    ax2.set_ylabel("Real-Time Factor")
    ax2.set_title("Processing Speed (lower is better)")
    ax2.legend()
    ax2.tick_params(axis="x", rotation=45)

# Plot 3: Memory Comparison
if "Memory (MB)" in df_comparison.columns:
    ax3 = axes[1, 0]
    for model in df_comparison["Model"].unique():
        model_data = df_comparison[df_comparison["Model"] == model]
        ax3.bar(model_data["Test"], model_data["Memory (MB)"], label=model, alpha=0.7)
    ax3.set_ylabel("Memory (MB)")
    ax3.set_title("Memory Usage (lower is better)")
    ax3.legend()
    ax3.tick_params(axis="x", rotation=45)

# Plot 4: Production Scores
if len(df_scores) > 0:
    ax4 = axes[1, 1]
    for model in df_scores["Model"].unique():
        model_data = df_scores[df_scores["Model"] == model]
        ax4.bar(model_data["Test"], model_data["Production Score"], label=model, alpha=0.7)
    ax4.axhline(y=80, color="g", linestyle="--", label="Production ready")
    ax4.axhline(y=60, color="orange", linestyle="--", label="Monitor")
    ax4.set_ylabel("Score (0-100)")
    ax4.set_title("Production Readiness Score")
    ax4.legend()
    ax4.tick_params(axis="x", rotation=45)

plt.tight_layout()

# Save figure
plot_path = results_dir / "model_comparison_plots.png"
plt.savefig(plot_path, dpi=150, bbox_inches="tight")
print(f"✓ Comparison plots saved to {plot_path}")

plt.show()

In [ ]:
# === COST ANALYSIS ===

print("=== Cost-Performance Analysis ===")

# Model specifications
model_specs = {
    "LFM2.5-Audio": {
        "parameters": "1.5B",
        "memory_mb": df_comparison[df_comparison["Model"] == "LFM2.5-Audio"]["Memory (MB)"].mean()
        if len(df_comparison) > 0
        else 0,
        "avg_rtf": df_comparison[df_comparison["Model"] == "LFM2.5-Audio"]["RTF"].mean()
        if len(df_comparison) > 0
        else 0,
        "avg_wer": df_comparison[df_comparison["Model"] == "LFM2.5-Audio"]["WER (%)"].mean()
        if len(df_comparison) > 0
        else 0,
    },
    "Whisper-Large-V3": {
        "parameters": "1.5B",
        "memory_mb": df_comparison[df_comparison["Model"] == "Whisper-Large-V3"][
            "Memory (MB)"
        ].mean()
        if len(df_comparison) > 0
        else 0,
        "avg_rtf": df_comparison[df_comparison["Model"] == "Whisper-Large-V3"]["RTF"].mean()
        if len(df_comparison) > 0
        else 0,
        "avg_wer": df_comparison[df_comparison["Model"] == "Whisper-Large-V3"]["WER (%)"].mean()
        if len(df_comparison) > 0
        else 0,
    },
}

# Calculate cost metrics
for model_name, specs in model_specs.items():
    print(f"\n{model_name}:")
    print(f"  Parameters: {specs['parameters']}")
    print(f"  Memory: {specs['memory_mb']:.1f} MB")
    print(f"  Speed (RTF): {specs['avg_rtf']:.3f}x")
    print(f"  Accuracy (WER): {specs['avg_wer']:.1f}%")

    # Cost per hour of processing
    if specs["avg_rtf"] > 0:
        hours_per_hour = 1 / specs["avg_rtf"]
        print(f"  Processing capacity: {hours_per_hour:.1f}x realtime")

    # Accuracy-efficiency score
    if specs["avg_wer"] > 0 and specs["avg_rtf"] > 0:
        efficiency = (100 - specs["avg_wer"]) / specs["avg_rtf"]
        print(f"  Efficiency score: {efficiency:.1f}")

# Recommendation based on use case
print("\n=== Use Case Recommendations ===")
print("Real-time applications: LFM2.5-Audio (better speed/accuracy balance)")
print("Batch processing: Whisper-Large-V3 (better accuracy, slower)")
print("Memory-constrained: LFM2.5-Audio (smaller footprint)")
print("Best overall: Based on your production requirements")

## 🎯 **Decision Summary**

This dashboard transforms experimental results into actionable production decisions:

### **Key Metrics**:
- **WER**: Word Error Rate (lower = better accuracy)
- **RTF**: Real-Time Factor (lower = faster, <1.0 = realtime)
- **Memory**: RAM usage (lower = better for deployment)
- **Production Score**: Combined metric (0-100, higher = better)

### **Next Steps**:
1. Run all evaluation notebooks to generate results
2. Review comparative scorecard above
3. Check production readiness grades
4. Use cost analysis for deployment planning

### **Production Decision Tree**:
- Score ≥ 80: ✅ Deploy with confidence
- Score 60-80: ⚠️ Deploy with monitoring
- Score < 60: ❌ Not production-ready